In [2]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

open_ai_key = os.getenv("OPEN_AI_KEY")
google_search_key = os.getenv("GOOGLE_PROGRAMMABLE_SEARCH_KEY")

In [5]:
dfs = []
for i in range(1, 11):
    df = pd.read_csv(f'data/asap_{i:02d}/df.csv')
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)

In [ ]:
import nltk
from nltk import ngrams, pos_tag
from nltk.tokenize import word_tokenize
from collections import Counter

def generate_ngram_results(set_ids, df, count_threshold = 3):
    # Initialize dictionary to store results
    ngram_results = {i: [] for i in set_ids}

    # Loop through each EssaySet
    for set_id in set_ids:
        # Filter DataFrame for current EssaySet and score1 == 2
        filtered_df = df[(df['EssaySet'] == set_id) & (df['Score1'] == 2)]
        
        # Print progress for each EssaySet
        print(f"Processing EssaySet {set_id} with {len(filtered_df)} essays.")
        
        # Initialize a Counter to count n-grams across all essays in the set
        ngram_counter = Counter()
        
        # Process each EssayText in the filtered_df DataFrame
        for essay in filtered_df['EssayText']:
            tokens = word_tokenize(essay)
            pos_tags = pos_tag(tokens)
            
            # Generate bi-grams, tri-grams, and tetra-grams
            bi_grams = list(ngrams(pos_tags, 2))
            tri_grams = list(ngrams(pos_tags, 3))
            tetra_grams = list(ngrams(pos_tags, 4))
            
            # Concatenate all n-grams into a single list and update the counter
            all_ngrams = bi_grams + tri_grams + tetra_grams
            ngram_counter.update(all_ngrams)
        
        # Filter n-grams that appeared at least 3 times
        frequent_ngrams = [ngram for ngram, count in ngram_counter.items() if count >= count_threshold]
        
        # Store the frequent n-grams in the results dictionary
        ngram_results[set_id] = frequent_ngrams
        
        # Print after processing each set
        print(f"Completed processing EssaySet {set_id}, with {len(frequent_ngrams)} n-grams.")
    
    return ngram_results


setup

In [278]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('word2vec_model/GoogleNews-vectors-negative300.bin', binary=True)  

In [ ]:
from gensim.models.doc2vec import Doc2Vec
doc2vec_model = Doc2Vec.load("doc2vec_model/doc2vec_wiki_d300_n5_w8_mc50_t12_e10_dbow.model")

In [279]:
import nltk
from nltk import ngrams, pos_tag
from nltk.tokenize import word_tokenize
from collections import Counter

def generate_ngram_results(set_ids, df, count_threshold = 3):
    # Initialize dictionary to store results
    ngram_results = {i: [] for i in set_ids}

    # Loop through each EssaySet
    for set_id in set_ids:
        # Filter DataFrame for current EssaySet and score1 == 2
        filtered_df = df[(df['EssaySet'] == set_id) & (df['Score1'] == 2)]
        
        # Print progress for each EssaySet
        print(f"Processing EssaySet {set_id} with {len(filtered_df)} essays.")
        
        # Initialize a Counter to count n-grams across all essays in the set
        ngram_counter = Counter()
        
        # Process each EssayText in the filtered_df DataFrame
        for essay in filtered_df['EssayText']:
            tokens = word_tokenize(essay)
            pos_tags = pos_tag(tokens)
            
            # Generate bi-grams, tri-grams, and tetra-grams
            bi_grams = list(ngrams(pos_tags, 2))
            tri_grams = list(ngrams(pos_tags, 3))
            tetra_grams = list(ngrams(pos_tags, 4))
            
            # Concatenate all n-grams into a single list and update the counter
            all_ngrams = bi_grams + tri_grams + tetra_grams
            ngram_counter.update(all_ngrams)
        
        # Filter n-grams that appeared at least 3 times
        frequent_ngrams = [ngram for ngram, count in ngram_counter.items() if count >= count_threshold]
        
        # Store the frequent n-grams in the results dictionary
        ngram_results[set_id] = frequent_ngrams
        
        # Print after processing each set
        print(f"Completed processing EssaySet {set_id}, with {len(frequent_ngrams)} n-grams.")
    
    return ngram_results


In [353]:
ngram_results = generate_ngram_results([1,2,3,4,5,6,7,8,9,10], df, 30)

Processing EssaySet 1 with 886 essays.
Completed processing EssaySet 1, with 459 n-grams.
Processing EssaySet 2 with 751 essays.
Completed processing EssaySet 2, with 426 n-grams.
Processing EssaySet 3 with 699 essays.
Completed processing EssaySet 3, with 403 n-grams.
Processing EssaySet 4 with 230 essays.
Completed processing EssaySet 4, with 72 n-grams.
Processing EssaySet 5 with 73 essays.
Completed processing EssaySet 5, with 13 n-grams.
Processing EssaySet 6 with 126 essays.
Completed processing EssaySet 6, with 28 n-grams.
Processing EssaySet 7 with 723 essays.
Completed processing EssaySet 7, with 327 n-grams.
Processing EssaySet 8 with 1313 essays.
Completed processing EssaySet 8, with 845 n-grams.
Processing EssaySet 9 with 1031 essays.
Completed processing EssaySet 9, with 563 n-grams.
Processing EssaySet 10 with 970 essays.
Completed processing EssaySet 10, with 583 n-grams.


In [281]:
import os

def load_prompts(set_ids):
    prompts = {}
    for set_id in set_ids:
        file_name = f"prompts/asap_{set_id:02d}.txt"
        if os.path.exists(file_name):
            with open(file_name, 'r') as file:
                prompts[set_id] = file.read().strip()
        else:
            print(f"Prompt file {file_name} not found.")
    return prompts

set_ids = list(range(1, 11))  # Example set ids, you can modify this as needed
prompts = load_prompts(set_ids)


In [282]:
import os

def load_only_prompts(set_ids):
    prompts = {}
    for set_id in set_ids:
        file_name = f"only_prompts/asap_{set_id:02d}.txt"
        if os.path.exists(file_name):
            with open(file_name, 'r') as file:
                prompts[set_id] = file.read().strip()
        else:
            print(f"Prompt file {file_name} not found.")
    return prompts

only_prompts = load_only_prompts(list(range(1, 11)))


In [283]:
import requests
import tiktoken
from bs4 import BeautifulSoup
from openai import OpenAI
client = OpenAI(api_key=open_ai_key)
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
import json
import concurrent.futures

def extract_articles(keyword):
      url = 'https://customsearch.googleapis.com/customsearch/v1'
      params = {
          'key': google_search_key,
          'cx': '901399343dffc41a5',
          'q': keyword,
          'start': 1,
          'num': 10,
          'lr': "lang_en",
          "filter": 1
      }      
      articles = []
      all_links = []
      try:
        for i in range(2):
          response = requests.get(url, params={
              **params,
              "start": i * 10 + 1
          })
          data = response.json()
          links = [item['link'] for item in data['items'] if 'link' in item]
          all_links.extend(links)

        print("All links: ", all_links)
        
        # Function to scrape a single link
        def scrape_link(link):
            try:
                page = requests.get(link, timeout=10)
                soup = BeautifulSoup(page.content, 'html.parser')
                return soup.get_text()
            except Exception as e:
                print(f"Failed to scrape {link}: {e}")
                return None

        # Use ThreadPoolExecutor to scrape links concurrently
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = list(executor.map(scrape_link, all_links))

        # Filter out None results and extend articles list
        articles.extend([result for result in results if result is not None])

      except Exception as e:
          print(f"An error occurred: {e}")
          # all.extend(data["items"])
          
      return articles

def extract_domain_specific_keywords(prompt, answers):
    # first 50k tokens
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(answers)
    first_50000_tokens = tokens[:50000]
    extracted_text = tokenizer.decode(first_50000_tokens)

    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": f"Here is a question: {prompt}. Identify 100 domain-specific keywords that are most relevant to the question from the following set of student answers, and return them in a JSON object with an array field named 'keywords':\n\n{extracted_text}"}
        ]
    )
    
    keywords = json.loads(response.choices[0].message.content)

    return keywords

def extract_tfidf_from_articles(keyword, articles):
    def preprocess(text):
        return text.lower()

    # Preprocess articles
    preprocessed_articles = [preprocess(article) for article in articles]

    # Initialize the vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the articles
    tfidf_matrix = vectorizer.fit_transform(preprocessed_articles)

    # Get the feature names (i.e., the words)
    feature_names = vectorizer.get_feature_names_out()

    # Split the keyword into individual words and lower each
    keywords = keyword.lower().split()

    # Initialize list to store tf-idf values for each word in the keyword
    tfidf_values_list = []

    # Iterate over each word in the keyword
    for word in keywords:
        try:
            # Find the index of the word
            word_index = feature_names.tolist().index(word)
            # Get the tf-idf values for the word across all documents
            tfidf_values = tfidf_matrix[:, word_index].toarray().flatten()
            # Append the average tf-idf value for the word
            tfidf_values_list.append(tfidf_values.mean())
        except ValueError:
            # If the word is not found, append 0.0
            tfidf_values_list.append(0.0)

    # Calculate the average tf-idf value for the entire keyword phrase
    if tfidf_values_list:
        average_tfidf = sum(tfidf_values_list) / len(tfidf_values_list)
    else:
        average_tfidf = 0.0

    return average_tfidf


def extract_weighted_keywords(setnumbers, df, prompts):
  weighted_keywords = {}
  all_articles = {}
  for setno in setnumbers:
    answers = " ".join(df[df['EssaySet'] == setno]['EssayText'].tolist())
    keywords = extract_domain_specific_keywords(prompts[setno], answers)["keywords"]
    # keywords = ['animals', 'Asia', 'Australia', 'Bamboo', 'Bears', 'Biologists', 'Biodiversity', 'Boa constrictor', 'Burmese python', 'Bushnell', 'Cape Coral', 'Carnivores', 'Change', 'China', 'Climates', 'Cobra', 'Cold blooded', 'Creatures', 'Customer', 'Ecosystem', 'Ecophobia', 'Effective threat', 'Endangered species', 'Environment', 'Everglades', 'Exclusive food', 'Exploded', 'Food source', 'Florida', 'Food stability', 'Food variety', 'Generalists', 'Government', 'Habitat', 'Hankering', 'Herbivores', 'Hood spread', 'Households', 'Humans', 'Hurricane Andrew', 'Invasive species', 'Key Largo wood rat', 'Kimberly', 'Koala', 'Lizards', 'Macroherbivores', 'Macinnes', 'Madagascar', 'Mammals', 'Mcinnes', 'Microchip', 'Monitor lizard', 'Natural predators', 'Niche environments', 'Nile monitor lizard', 'Non-native', 'Organisms', 'Panda', 'Pandas', 'Permit', 'Pets', 'Plants', 'Policy adviser', 'Potomac', 'Predators', 'Prey', 'Python', 'Raccoon', 'Radiated tortoises', 'Reed', 'Reptiles', 'Resource-specific', 'RobRoy Macinnes', 'Scales', 'Scorpion', 'Sensitive habitats', 'Skip Snow', 'Specialists', 'Stability', 'Survival', 'Tourists', 'Traditional diet', 'Urban areas', 'U.S. Fish and Wildlife Service', 'U.S. Geological Survey', 'Unique habitat', 'Variety of foods', 'Warm blooded', 'Weeds', 'Wildlife biologist', 'Zoologists']
    
    set_scores = {}

    total_keywords = len(keywords)
    for i, keyword in enumerate(keywords, 1):
       articles = extract_articles(keyword)
       tfidf = extract_tfidf_from_articles(keyword, articles)
       
       set_scores[keyword] = tfidf
       print(f"Progress: {i}/{total_keywords} - keyword: {keyword}, tfidf: {tfidf}")
       
       if setno not in all_articles:
           all_articles[setno] = {}
       all_articles[setno][keyword] = articles
    
    weighted_keywords[setno] = set_scores
  
  return weighted_keywords, all_articles

weighted_keywords, all_articles = extract_weighted_keywords([1,2,3,4,5,6,7,8,9,10], df, prompts)

[nltk_data] Downloading package punkt to /Users/josephtey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


All links:  ['https://experiment.com/', 'https://experimentbeauty.com/', 'https://en.wikipedia.org/wiki/Experiment', 'https://labs.google/', 'https://www.merriam-webster.com/dictionary/experiment', 'https://atlas.cern/', 'https://experiments.withgoogle.com/collection/chrome', 'https://www.imdb.com/title/tt0250258/', 'https://aspe.hhs.gov/reports/overview-final-report-seattle-denver-income-maintenance-experiment', 'https://www.prisonexp.org/conclusion', 'https://www.wnycstudios.org/podcasts/experiment', 'https://www.cdc.gov/tuskegee/timeline.htm', 'https://cals.cornell.edu/agricultural-experiment-station', 'https://hbr.org/2010/04/column-why-businesses-dont-experiment', 'https://www.prisonexp.org/', 'https://support.google.com/google-ads/answer/6261395?hl=en', 'https://theexperimentpublishing.com/', 'https://www.uaf.edu/afes/places/fairbanks/', 'https://journals.ametsoc.org/view/journals/bams/93/4/bams-d-11-00094.1.xml', 'https://www.experiment.org/scholarships/']
Progress: 1/98 - keywo

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress: 2/98 - keyword: procedure, tfidf: 0.054361147770560184
All links:  ['https://www.mass.gov/', 'https://en.wikipedia.org/wiki/Mass', 'https://massmoca.org/', 'https://massdesigngroup.org/', 'https://www.masssave.com/', 'https://www.massaudubon.org/', 'https://massculturalcouncil.org/', 'https://masshumanities.org/', 'https://www.imdb.com/title/tt11389748/', 'https://www.nhlbi.nih.gov/health/educational/lose_wt/BMI/bmicalc.htm', 'https://www.massgeneralbrigham.org/en', 'https://mtc.dor.state.ma.us/mtc/', 'https://www.massmutual.com/', 'https://www.merriam-webster.com/dictionary/mass', 'https://masstimes.org/', 'https://www.massgeneral.org/', 'https://www.cdc.gov/healthyweight/assessing/bmi/adult_bmi/english_bmi_calculator/bmi_calculator.html', 'https://dtaconnect.eohhs.mass.gov/', 'https://www.ea.com/games/mass-effect/mass-effect-legendary-edition', 'https://www.gunviolencearchive.org/reports/mass-shooting']
Failed to scrape https://www.massmutual.com/: ('Connection aborted.', R

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Progress: 59/90 - keyword: absorb heat, tfidf: 0.0711148683043349
All links:  ['https://www.sciencelearn.org.nz/resources/48-reflection-of-light', 'https://www.olympus-lifescience.com/en/microscope-resource/primer/lightandcolor/reflectionintro/', 'https://www.reddit.com/r/askscience/comments/qtqut/why_do_animals_eyes_reflect_light_when_humans_dont/', 'https://blenderartists.org/t/how-do-you-make-an-emission-surface-also-reflect-light/1263292', 'https://www.reddit.com/r/PokemonVGC/comments/13ryz57/reflect_light_screen_duration_increase/', 'https://blender.stackexchange.com/questions/153131/blender-2-8-how-do-i-make-a-texture-reflect-light', 'https://forum.powerscore.com/viewtopic.php?t=1804', 'https://mercymultiplied.com/reflect-the-light-of-christ/', 'https://forum.keyshot.com/index.php?topic=28418.0', 'https://worldbuilding.stackexchange.com/questions/209175/would-a-moon-with-plantlife-still-give-off-reflect-light', 'https://byjus.com/physics/reflection-of-light/', 'https://austinkleo

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress: 77/90 - keyword: dark color absorb, tfidf: 0.03263006543331085
All links:  ['https://www.reddit.com/r/explainlikeimfive/comments/d9h4uv/elif_why_do_dark_colors_attract_more_heat/', 'https://www.terminix.com/mosquitoes/what-attracts-mosquitoes/are-mosquitoes-attracted-to-color/', 'https://www.reddit.com/r/INTP/comments/l2x1tx/are_intps_naturally_attracted_to_dark_colors_and/', 'https://www.jcehrlich.com/help-and-advice/blog/mosquitoes/mosquitoes-and-the-color-red', 'https://mosquitojoe.com/blog/what-colors-attract-mosquitoes/', 'https://mosquitonix.com/blogs/news/are-mosquitos-attracted-to-black', 'https://healthcare.utah.edu/huntsmancancerinstitute/news/2023/08/what-clothing-best-sun-protection', 'https://www.quora.com/Do-darker-colored-cats-attract-more-fleas-than-lighter-colored-cats', 'https://www.cnn.com/2016/04/25/health/bedbugs-dark-color-sheets/index.html', 'https://www.quora.com/Is-it-true-that-blonde-guys-are-attracted-to-dark-haired-girls-and-vice-versa', 'https://w

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to scrape http://www.allpaintingandrestoration.com/are-your-exterior-house-painting-colors-attracting-pests-in-fountain-hills-az-dark-paint-colors-attract-mosquitoes-yellow-orange-bugs-etc/: HTTPConnectionPool(host='www.allpaintingandrestoration.com', port=80): Read timed out. (read timeout=10)
Progress: 78/90 - keyword: dark color attract, tfidf: 0.02418397994856201
All links:  ['https://www.reddit.com/r/k9sports/comments/uzwhdm/how_to_keep_dog_cool_when_leaving_him_in_the_car/', 'https://www.quora.com/What-are-good-ways-to-keep-my-dog-cool-at-home-in-summer-while-Im-away-at-work', 'https://www.reddit.com/r/puppy101/comments/1538nvl/how_do_you_keep_your_dog_cool/', 'https://outdoors.stackexchange.com/questions/9493/how-do-you-keep-your-dog-cool-while-hiking-on-hot-sunny-days', 'https://www.quora.com/The-air-conditioning-broke-and-my-dog-is-hot-How-do-I-cool-him-down-until-its-fixed', 'https://theocbarkyard.wordpress.com/2015/04/12/how-to-keep-your-dog-cool-and-comfortable-in-th

In [284]:
import json

# Define the file paths
weighted_keywords_file = "weighted_keywords.json"
all_articles_file = "all_articles.json"

# Dump weighted_keywords into a json file
with open(weighted_keywords_file, "w") as f:
    json.dump(weighted_keywords, f)

# Dump all_articles into a json file
with open(all_articles_file, "w") as f:
    json.dump(all_articles, f)


In [285]:
from utils.feature_extractor import FeatureExtractor

feature_extractor = FeatureExtractor(prompts, only_prompts, model, doc2vec_model, ngram_results, weighted_keywords)

train model

In [471]:
from numpy import hstack

# run this for everythting ... so i can just do it once, and then save it as a csv to re-load ... 
def add_features_to_df(df, feature_extractor, add_word2vec=False, add_doc2vec=False, add_pos=False, add_prompt_overlap=False, add_weighted_keywords=False, add_lexical_overlap=False, add_stylistic_features=False, add_logical_operators=False, add_temporal_features=False):
    if add_word2vec:
        df['word2vec_features'] = None
    if add_doc2vec:
        df['doc2vec_features'] = None
    if add_pos:
        df['pos_features'] = None
    if add_prompt_overlap:
        df['prompt_overlap_features'] = None
    if add_weighted_keywords:
        df['weighted_keywords_features'] = None
    if add_lexical_overlap:
        df['lexical_overlap_features'] = None
    if add_stylistic_features:
        df['stylistic_features'] = None
    if add_logical_operators:
        df['logical_operators_features'] = None
    if add_temporal_features:
        df['temporal_features'] = None

    total_items = len(df)
    print(f"Processing {total_items} items...")

    # Iterate over each row in the training DataFrame
    num = 0
    for index, row in df.iterrows():
        # Extract features using the feature_extractor object
        if add_word2vec:
            word2vec_features = feature_extractor.word2vec(row['EssayText'])
            df.at[index, 'word2vec_features'] = word2vec_features
        if add_doc2vec:
            doc2vec_features = feature_extractor.doc2vec(row['EssayText'])
            df.at[index, 'doc2vec_features'] = doc2vec_features
        if add_pos:
            pos_features = feature_extractor.pos(row['EssaySet'], row['EssayText'])
            df.at[index, 'pos_features'] = pos_features
        if add_prompt_overlap:
            prompt_overlap_features = feature_extractor.prompt_overlap(row['EssaySet'], row['EssayText'])
            df.at[index, 'prompt_overlap_features'] = prompt_overlap_features
        if add_weighted_keywords:
            weighted_keywords_features = feature_extractor.weighted_domain_specific_keywords(row['EssaySet'], row['EssayText'])
            df.at[index, 'weighted_keywords_features'] = weighted_keywords_features
        if add_lexical_overlap:
            lexical_overlap_features = feature_extractor.lexical_overlap(row['EssaySet'], row['EssayText'])
            df.at[index, 'lexical_overlap_features'] = lexical_overlap_features
        if add_stylistic_features:
            stylistic_features = feature_extractor.stylistic_features(row['EssayText'])
            df.at[index, 'stylistic_features'] = stylistic_features
        if add_logical_operators:
            logical_operators_features = feature_extractor.logical_operators(row['EssayText'])
            df.at[index, 'logical_operators_features'] = logical_operators_features
        if add_temporal_features:
            temporal_features = feature_extractor.temporal_features(row['EssayText'])
            df.at[index, 'temporal_features'] = temporal_features

        # Update after every 1000 items
        if (num) % 1000 == 0:
            print(f"Processed {num} items.")
        num += 1

    return df

In [24]:
highest_score = {1: 3, 2: 3, 3: 2, 4: 2, 5: 3, 6: 3, 7: 2, 8: 2, 9: 2, 10: 2}
for essay_set in range(1, 11):
    essay_set_df = df[df['EssaySet'] == essay_set].copy(deep=True)
    augment_dataset(essay_set_df, essay_set)

In [23]:
import random
def augment_dataset(essay_set_df, essay_set):
    # dataset augmentation
    additional_training_data = []
    for es in list(range(1, 11)) + ([essay_set]*10):
        es_df = df[df['EssaySet'] == es]
        highest_score_value = highest_score[es]
        highest_score_responses = es_df[(es_df['Score1'] == highest_score_value) & (es_df['Score2'] == highest_score_value) & (es_df['split'] == "train")]

        if len(highest_score_responses) > 0:
            sampled_response = highest_score_responses.sample(n=1, random_state=42)
            if es == essay_set:
                essay_text = sampled_response['EssayText'].values[0]
                jumbled_text = ' '.join(random.sample(essay_text.split(), len(essay_text.split())))
                sampled_response['EssayText'] = jumbled_text
            sampled_response['EssaySet'] = essay_set
            sampled_response['CorrectedSpellingEssayText'] = sampled_response['EssayText']
            sampled_response['Score1'] = 0
            sampled_response['Score2'] = 0
            sampled_response['split_custom'] = sampled_response['split']
            additional_training_data.append(sampled_response)

    pd.concat(additional_training_data, ignore_index=True).to_csv(f'augmented_training_data/augmented_training_data_{essay_set}.csv', index=False)

    return pd.concat([essay_set_df] + additional_training_data, ignore_index=True)


In [503]:
# Save the updated DataFrame to a CSV file
import os
from datetime import datetime

augment_dataset(essay_set_df, essay_set)

# Execute the function to add features to the DataFrame and display the updated DataFrame
add_features_to_df(essay_set_df, feature_extractor, add_word2vec=True, add_doc2vec=True, add_pos=True, add_prompt_overlap=True, add_weighted_keywords=True, add_lexical_overlap=True, add_stylistic_features=True, add_logical_operators=True, add_temporal_features=True)

# Create the folder if it doesn't exist
if not os.path.exists('updated_features'):
    os.makedirs('updated_features')

# Get the current timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Save the updated DataFrame to a CSV file with the timestamp
essay_set_df.to_csv(f'updated_features/updated_features_{timestamp}.csv', index=False)


Processing 3452 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processed 3000 items.


In [510]:
# Save the updated DataFrame to a CSV file
import os
from datetime import datetime

# Initialize a dictionary to store DataFrames for each essay set
essay_set_dfs = {}

# Loop through essay sets from 1 to 10
for essay_set in range(1, 11):
    essay_set_df = df[df['EssaySet'] == essay_set].copy(deep=True)

    # Dataset Augmentation
    augment_dataset(essay_set_df, essay_set)

    # Execute the function to add features to the DataFrame
    add_features_to_df(essay_set_df, feature_extractor, add_word2vec=True, add_doc2vec=True, add_pos=True, add_prompt_overlap=True, add_weighted_keywords=True, add_lexical_overlap=True, add_stylistic_features=True, add_logical_operators=True, add_temporal_features=True)
    
    # Store the updated DataFrame in the dictionary
    essay_set_dfs[essay_set] = essay_set_df
    
    # Create the folder if it doesn't exist
    if not os.path.exists('updated_features'):
        os.makedirs('updated_features')
    
    # Get the current timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Save the updated DataFrame to a CSV file with the timestamp
    essay_set_df.to_csv(f'updated_features/updated_features_{essay_set}_{timestamp}.csv', index=False)

Processing 2787 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processing 2130 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processing 3152 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processed 3000 items.
Processing 2897 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processing 2992 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processing 2995 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processing 2999 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processing 2999 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processing 2997 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processing 2734 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.


In [511]:
import pickle

# Save the dictionary of DataFrames to a pickle file
with open(f'essay_set_dfs/essay_set_dfs_{timestamp}.pkl', 'wb') as f:
    pickle.dump(essay_set_dfs, f)

In [512]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# features = ['pos_features', 'prompt_overlap_features', 'weighted_keywords_features', 'lexical_overlap_features', 'stylistic_features', 'logical_operators_features', 'temporal_features']
def test_model(essay_set, essay_set_dfs, features, verbose=False):
    essay_set_df = essay_set_dfs[essay_set]

    # prepare data set 
    scaler = MinMaxScaler(feature_range=(0, 1))

    # generate training data 
    training_data = essay_set_df[(essay_set_df['DataSet'] == 'Train')].copy(deep=True)
    training_data = training_data.dropna(subset=['word2vec_features', 'doc2vec_features'])
    X_train = np.vstack(training_data[features].apply(lambda x: np.hstack(x), axis=1))
    X_train = scaler.fit_transform(X_train)
    y_train = training_data['Score1']

    num_features = X_train.shape[1]
    print(f"Number of features in X_train: {num_features}")

    # Define hyperparameter ranges
    max_depths = [50, 75, 100, 150, 200]
    n_estimators_list = [50, 75, 100, 150, 200]

    # Initialize variables to store the best model and its score
    best_kappa_score = -1
    best_params = None
    best_clf = None

    # Initialize a list to store results
    results = []

    # Iterate over all combinations of hyperparameters
    for max_depth in max_depths:
        for n_estimators in n_estimators_list:
            # Train model
            clf = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators)
            clf.fit(X_train, y_train)
            
            # Validate model
            val_set = essay_set_df[(essay_set_df['DataSet'] == 'Priva')].copy(deep=True)
            val_set = val_set.dropna(subset=['word2vec_features', 'doc2vec_features'])
            
            X_val = np.vstack(val_set[features].apply(lambda x: np.hstack(x), axis=1).values)
            X_val = scaler.transform(X_val)
            y_val = val_set['Score1']
            
            y_pred = clf.predict(X_val)
            
            # Calculate evaluation metrics
            conf_matrix = confusion_matrix(y_val, y_pred)
            kappa_score = cohen_kappa_score(y_val, y_pred, weights='quadratic', sample_weight=None)
            accuracy = accuracy_score(y_val, y_pred)
            
            # Print the results for the current hyperparameters
            if verbose:
              print(f"Max Depth: {max_depth}, N Estimators: {n_estimators}")
              print(f"Cohen Kappa Score (Weighted): {kappa_score}")
              print(f"Accuracy: {accuracy}")
            
            # Append the results to the list
            results.append({
                'max_depth': max_depth,
                'n_estimators': n_estimators,
                'qwk': kappa_score,
                'accuracy': accuracy
            })
            
            # Update the best model if the current one is better
            if kappa_score > best_kappa_score:
                best_kappa_score = kappa_score
                best_params = (max_depth, n_estimators)
                best_clf = clf

    # Print the best hyperparameters and their score
    print(f"Essay Set: {essay_set}")
    print(f"Best Hyperparameters: Max Depth: {best_params[0]}, N Estimators: {best_params[1]}")
    print(f"Best Cohen Kappa Score (Weighted): {best_kappa_score}")

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Get the current timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Save the results to a CSV file
    results_df.to_csv(f'{essay_set}_results_{timestamp}_{"_".join(features)}.csv', index=False)

    return best_clf


In [513]:
features = ['pos_features', 'prompt_overlap_features', 'weighted_keywords_features', 'lexical_overlap_features', 'stylistic_features', 'logical_operators_features', 'temporal_features']

best_models = {}
for essay_set in range(1, 11):
    best_models[essay_set] = test_model(essay_set, essay_set_dfs, features)

Number of features in X_train: 593
Essay Set: 1
Best Hyperparameters: Max Depth: 75, N Estimators: 150
Best Cohen Kappa Score (Weighted): 0.702013385848459
Number of features in X_train: 542
Essay Set: 2
Best Hyperparameters: Max Depth: 100, N Estimators: 200
Best Cohen Kappa Score (Weighted): 0.6251395070526697
Number of features in X_train: 540
Essay Set: 3
Best Hyperparameters: Max Depth: 200, N Estimators: 50
Best Cohen Kappa Score (Weighted): 0.2622528311870077
Number of features in X_train: 210
Essay Set: 4
Best Hyperparameters: Max Depth: 100, N Estimators: 150
Best Cohen Kappa Score (Weighted): 0.6464078034139936
Number of features in X_train: 119
Essay Set: 5
Best Hyperparameters: Max Depth: 50, N Estimators: 75
Best Cohen Kappa Score (Weighted): 0.7046103056902663
Number of features in X_train: 147
Essay Set: 6
Best Hyperparameters: Max Depth: 100, N Estimators: 75
Best Cohen Kappa Score (Weighted): 0.6029569403151103
Number of features in X_train: 479
Essay Set: 7
Best Hyper

In [514]:
features = ['word2vec_features', 'doc2vec_features', 'pos_features', 'prompt_overlap_features', 'weighted_keywords_features', 'lexical_overlap_features', 'stylistic_features', 'logical_operators_features', 'temporal_features']

best_models = {}
for essay_set in range(1, 11):
    best_models[essay_set] = test_model(essay_set, essay_set_dfs, features)

Number of features in X_train: 1193
Essay Set: 1
Best Hyperparameters: Max Depth: 200, N Estimators: 150
Best Cohen Kappa Score (Weighted): 0.7161071764278867
Number of features in X_train: 1142
Essay Set: 2
Best Hyperparameters: Max Depth: 75, N Estimators: 150
Best Cohen Kappa Score (Weighted): 0.5529111649007871
Number of features in X_train: 1140
Essay Set: 3
Best Hyperparameters: Max Depth: 200, N Estimators: 150
Best Cohen Kappa Score (Weighted): 0.2281011000019264
Number of features in X_train: 810
Essay Set: 4
Best Hyperparameters: Max Depth: 100, N Estimators: 75
Best Cohen Kappa Score (Weighted): 0.6232305604033352
Number of features in X_train: 719
Essay Set: 5
Best Hyperparameters: Max Depth: 50, N Estimators: 75
Best Cohen Kappa Score (Weighted): 0.6386815044372447
Number of features in X_train: 747
Essay Set: 6
Best Hyperparameters: Max Depth: 50, N Estimators: 75
Best Cohen Kappa Score (Weighted): 0.5983764319855682
Number of features in X_train: 1079
Essay Set: 7
Best H